# Imports

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
display(pd.__version__)
display(mpl.__version__)

ModuleNotFoundError: No module named 'matplotlib'

# Methods

In [ ]:
match = self.bag_df.loc[self.bag_df.postcode == item['postcode'] & 
                        self.bag_df.huisnummer == item['huisnummer'] &
                        ((self.bag_df.huisletter == item['huisletter']) | (pd.isna(self.bag_df.huisletter) == pd.isna(item['huisletter']))) &
                        ((self.bag_df.toevoeging == item['toevoeging']) | (pd.isna(self.bag_df.huisletter) == pd.isna(item['toevoeging'])))
                       ].to_dict(
                orient='list')

In [4]:
def to_lower(x):
        return x.lower()

In [ ]:
def print_unique_values(column):
    lijst = []
    for s in list(jaap_bag[column].unique()):
        for i in str(s).split(', '):
            lijst.append(i)
        for s in list(set(lijst)):
            print(s)

In [25]:
def nullvalues(df):
    nullcolumns = df.notnull().sum().to_frame().reset_index()
    nullcolumns.columns = ['columns','non-null']
    nullcolumns['percent'] = nullcolumns['non-null'].apply(lambda x: round(x / len(df) * 100,2) if len(df) > 0 else 0)
    nullcolumns = nullcolumns.sort_values(by=['non-null','columns'], ascending=False).reset_index(drop=True)
    return nullcolumns

In [6]:
def bag_adres(pc='', huisnummer=0, straat='', plaats=''):
    if huisnummer != 0 and pc != '':
        return(bag.loc[(bag.postcode_BAG == pc) & (bag.huisnummer_BAG == huisnummer)])
    if huisnummer == 0 and pc != '':
        return(bag.loc[(bag.postcode_BAG == pc)])
    if straat != '' and plaats != '':
        return(bag.loc[(bag.straat_BAG.str.contains(straat)) & (bag.plaats_BAG == plaats)])
    if straat != '' and plaats == '':
        return(bag.loc[bag.straat_BAG.str.contains(straat)])

In [ ]:
def decode_str(input):
    return bytes(input, 'latin1').decode('utf-8', 'ignore')

In [ ]:
def match(df, bag_df):
    df_bag1 = df.merge(
        bag_df,
        on=['postcode', 'huisnummer', 'huisletter'],
        how='left',
        suffixes=('', '_BAG')
    )

    df_bag2 = df_bag1.loc[df_bag1['index'].isna()].drop(
            ['straat_BAG', 'plaats_BAG', 'gemeente', 'provincie_BAG', 'index'],
            axis=1).merge(
                            bag_df,
                            on=['postcode', 'huisnummer'],
                            how='left',
                            suffixes=('', '_BAG')
                            )

    df_bag1.set_index('jaap_id', inplace=True)
    df_bag2.set_index('jaap_id', inplace=True)
    df_bag1.update(df_bag2)

    df_bag1.reset_index(inplace=True)
    df_bag1.set_index(['postcode', 'huisnummer', 'huisletter', 'toevoeging',
                       'scrape_datum', 'bron'], inplace=True, verify_integrity=True)
    return df_bag1

In [ ]:
def not_match(df, column):
    res = len(df.loc[df[column].isna()])/len(df)*100
    print('{:.2f}%'.format(res))

In [ ]:
def read_data(path):
    data = pd.DataFrame()
    for f in os.listdir(path):
        file = pd.read_csv('../jaap/output/' + f, encoding='latin1')
        data = pd.concat([data, file])
    return data

In [1]:
class JaapSpider(scrapy.Spider):

    name = 'jaap'
    allowed_domains = ['jaap.nl']

    def __init__(self, start_url_list, *args, **kwargs):
        super(JaapSpider, self).__init__(*args, **kwargs)
        self.start_urls = ['https://www.jaap.nl/koophuizen/' + i for i in start_url_list]

    def parse(self, response):
        # get list of links from search results
        search_result = response.xpath('//a[@class="property-inner"]/@href').extract()

        # follow link in search results and yield item (house) from datapage
        for link in search_result:
            yield response.follow(link, callback=self.parse_features)

        # goto next page in search until no more pages
        next_page = response.xpath('//a[@rel="next"]/@href').extract_first()
        if next_page != '#':
            yield response.follow(next_page, callback=self.parse)

# Read

In [ ]:
bag_cols = ['straat_BAG', 'huisnummer_BAG', 'huisletter_BAG',
                'toevoeging_BAG', 'postcode_BAG', 'plaats_BAG',
                'gemeente_BAG', 'provincie_BAG']
dtype = {'straat_BAG':      'str',
             'huisnummer_BAG':  'int',
             'huisletter_BAG':  'str',
             'toevoeging_BAG':  'str',
             'postcode_BAG':    'str',
             'plaats_BAG':      'str',
             'gemeente_BAG':    'str',
             'provincie_BAG':   'str'}
bag = pd.read_csv('../../bag_db/BAG_most_current.csv', header=None, names=bag_cols, dtype=dtype, engine='c').reset_index()
bag['huisletter_BAG'] = bag['huisletter_BAG'].apply(lambda h: str(h).lower() if pd.notna(h) else 'NULL')
bag['toevoeging_BAG'] = bag['toevoeging_BAG'].apply(lambda h: str(h).lower() if pd.notna(h) else 'NULL')
bag.rename(columns={'index': 'index_BAG'}, inplace=True)
bag.drop_duplicates(subset=['postcode_BAG', 'huisnummer_BAG', 'huisletter_BAG', 'toevoeging_BAG'], keep='first', inplace=True)

In [5]:
def read_jaap(path):
    dtype = {
        'straat':                       'str',
        'status':                       'str',
        'huisnummer':                   'int',
        'huisletter':                   'str',
        'toevoeging':                   'str',
        'postcode':                     'str',
        'plaats':                       'str',
        'provincie':                    'str',
        'huidige_vraagprijs':           'float',
        'oorspr_vraagprijs':            'float',
        'prijs_wijzigingen_data':       'str',
        'prijs_wijzigingen_prijzen':    'str',
        'prijs_per_m2':                 'float',
        'tijd_in_de_verkoop':           'str',
        'bouwjaar':                     'float',
        'oppervlakte':                  'float',
        'perceeloppervlakte':           'float',
        'inhoud':                       'float',
        'aantal_kamers':                'float',
        'aantal_slaapkamers':           'float',
        'bijzonderheden':               'str',
        'isolatie':                     'str',
        'verwarming':                   'str',
        'jaap_id':                      'int'
    }
    jaap_df = pd.read_csv(path, encoding='windows-1251', engine='c', dtype=dtype)
    jaap_df.fillna({'huisletter': '-NULL-', 'toevoeging': '-NULL-'}, inplace=True)
    jaap['huisletter_BAG'] = bag['huisletter_BAG'].apply(lambda h: str(h).lower() if pd.notna(h) else '-NULL-')
    jaap['toevoeging_BAG'] = bag['toevoeging_BAG'].apply(lambda h: str(h).lower() if pd.notna(h) else '-NULL-')
    return jaap_df

In [2]:
df = pd.read_csv('../../output/dropped_test.csv') 

In [3]:
len(df)

7728

In [4]:
df.loc[(df.toevoeging == 'ong') | df.straat.str.contains('ong', case=False)].sort_values('plaats')

,straat,huisnummer,huisletter,toevoeging,postcode,plaats,huidige_vraagprijs,oorspr_vraagprijs,prijs_wijzigingen_data,prijs_per_m2,...,aantal_slaapkamers,inhoud,tuin,garage,energielabel,verwarming,isolatie,bron,laatste_scrape,jaap_id
4406,Anna Van Buerenplein,NaN,NaN,ong,2595DE,'S-Gravenhage,1650000.0,1650000.0,NaN,8462.0,...,3.0,NaN,NaN,Ja,NaN,NaN,NaN,jaap.nl,18-09-2018,15659517.0
4107,Keizerstraat Ong,NaN,NaN,NaN,NaN,'S-Gravenhage,39000.0,39000.0,12-02-2018/13-02-2018/18-09-2018|39000/39000/3...,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15592797.0
4011,Weidevogellaan,NaN,NaN,ong,2496MX,'S-Gravenhage,17500.0,17500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15566641.0
3990,Anna Blamanplein,NaN,NaN,ong,NaN,'S-Gravenhage,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,14885308.0
3382,Van Alkemadelaan,NaN,NaN,ong,2597AV,'S-Gravenhage,335000.0,335000.0,NaN,3045.0,...,2.0,390.0,NaN,Ja,NaN,NaN,NaN,jaap.nl,18-09-2018,15587328.0
3441,Boomaweg Ong,NaN,NaN,NaN,2553EB,'S-Gravenhage,564605.0,564605.0,NaN,3321.0,...,NaN,510.0,Ja,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15614005.0
3470,Abdis Johannastraat Ong,NaN,NaN,NaN,2544NT,'S-Gravenhage,563763.0,563763.0,NaN,3356.0,...,NaN,610.0,Ja,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15632146.0
3528,Anna Van Buerenplein Ong,NaN,NaN,NaN,NaN,'S-Gravenhage,1650000.0,1650000.0,NaN,8462.0,...,NaN,632.0,NaN,Ja,NaN,NaN,NaN,jaap.nl,18-09-2018,15659462.0
3805,Poniente,NaN,NaN,ong,NaN,'S-Gravenhage,819575.0,819575.0,NaN,4793.0,...,2.0,440.0,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15543556.0
3806,Poniente,NaN,NaN,ong,NaN,'S-Gravenhage,790900.0,790900.0,NaN,4625.0,...,2.0,440.0,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15543555.0


In [33]:
indx = df.loc[(df.straat.str.contains('kavel', case=False)) | (df.toevoeging.str.contains('ong', case=False)) | (df.straat.str.contains('ong', case=False)) | (df.straat.str.contains('bouw', case=False)) | 
              (df.straat.str.contains('fase', case=False)) | (df.soort_woning == 'Bouwgrond') | (df.toevoeging.str.contains('bouw', case=False))].index

In [34]:
bad_df = df.index.isin(indx)
df[~bad_df].sort_values('straat').to_excel('bad_df2.xlsx')

In [13]:
df.loc[df.soort_woning == 'Bouwgrond']

,straat,huisnummer,huisletter,toevoeging,postcode,plaats,huidige_vraagprijs,oorspr_vraagprijs,prijs_wijzigingen_data,prijs_per_m2,...,aantal_slaapkamers,inhoud,tuin,garage,energielabel,verwarming,isolatie,bron,laatste_scrape,jaap_id
36,Ursulineweg-Kavels Voor Vrijstaande Villa,NaN,NaN,NaN,NaN,Leusden,529000.0,529000.0,NaN,2645.0,...,NaN,600.0,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15653158.0
40,Venepad Kavel,4.0,NaN,ong,NaN,Oosterstreek,76521.0,76521.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15649088.0
41,Venepad Kavel,3.0,NaN,ong,NaN,Oosterstreek,54160.0,54160.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15649087.0
44,Venepad Kavel,1.0,NaN,ong,NaN,Oosterstreek,49223.0,49223.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15649081.0
76,Venepad Kavel,7.0,NaN,ong,NaN,Oosterstreek,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15649206.0
77,Venepad Kavel,8.0,NaN,ong,NaN,Oosterstreek,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15649205.0
97,Nije Boarnsterdyk,NaN,NaN,NaN,NaN,Akkrum,37500.0,37500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15645612.0
101,Jislumerdyk,NaN,NaN,ong,9111HC,Burdaard,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15591195.0
111,Zeedijk,NaN,NaN,NaN,8757JV,Gaast,26500.0,26500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,15596155.0
180,Nije Boarnsterdyk,NaN,NaN,NaN,NaN,Akkrum,37500.0,37500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jaap.nl,18-09-2018,14949764.0


# Merge

In [ ]:
jaap_bag = match(jaap_data, bag, drop_postcode=False)

In [ ]:
not_match(jaap_bag, 'gemeente')

In [ ]:
jaap_bag.loc[(slice(None), 12, slice(None), slice(None), 'Meijel'),:].T

# Tests

In [ ]:
bag.toevoeging.value_counts()

In [ ]:
bag_adres(pc='5768RJ')

# null columns

In [ ]:
nullvalues(jaap_bag)